<a href="https://colab.research.google.com/github/BianPaulParaguya/QQQ-Monitor/blob/main/QQQ_monitor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance
!pip install scikit-learn
!pip install pandas
!pip install xgboost
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 MB 4.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-cupti-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-nvrtc-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-runtime-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cudnn-cu12==8.9.2.26; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requ

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import datetime

def fetch_historical_data(ticker="aapl", period="1y"):
    stock = yf.Ticker(ticker)
    data = stock.history(period=period)
    return data

def compute_rsi(series, period=14):
    delta = series.diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def add_technical_indicators(data):
    data['SMA_10'] = data['Close'].rolling(window=10).mean()
    data['SMA_50'] = data['Close'].rolling(window=50).mean()
    data['RSI'] = compute_rsi(data['Close'])
    return data

def prepare_data(data):
    data = add_technical_indicators(data)
    data = data.dropna()

    data['Tomorrow'] = data['Close'].shift(-1)
    data['Target'] = (data['Tomorrow'] > data['Close']).astype(int)
    data = data.dropna()

    feature_columns = ['Close', 'Volume', 'SMA_10', 'SMA_50', 'RSI']
    features = data[feature_columns]
    targets = data['Target']

    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)

    X_train, X_test, y_train, y_test = train_test_split(features_scaled, targets, test_size=0.2, random_state=0)

    return (X_train, X_test, y_train, y_test), feature_columns

def train_model(features, targets):
    model = RandomForestClassifier(n_estimators=100, random_state=0)
    model.fit(features, targets)
    return model

def evaluate_model(model, X_test, y_test):
    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    report = classification_report(y_test, predictions)
    print(f"Accuracy: {accuracy:.2f}")
    print("Classification Report:\n", report)

def predict_swing(model, data, feature_columns, scaler):
    latest_data = data.iloc[-1][feature_columns].values.reshape(1, -1)
    latest_data_scaled = scaler.transform(latest_data)
    prediction = model.predict(latest_data_scaled)
    return prediction[0]

def main():
    data = fetch_historical_data()
    (X_train, X_test, y_train, y_test), feature_columns = prepare_data(data)

    model = train_model(X_train, y_train)
    evaluate_model(model, X_test, y_test)

    scaler = StandardScaler()
    scaler.fit(data[feature_columns])

    prediction = predict_swing(model, data, feature_columns, scaler)

    if prediction >= 0.5:
        print("Potential Upside Swing Predicted for QQQ")
    else:
        print("Potential Downside Swing Predicted for QQQ")

if __name__ == "__main__":
    main()

<ipython-input-1-87798ee82920>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Tomorrow'] = data['Close'].shift(-1)
<ipython-input-1-87798ee82920>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Target'] = (data['Tomorrow'] > data['Close']).astype(int)


Accuracy: 0.39
Classification Report:
               precision    recall  f1-score   support

           0       0.38      0.40      0.39        20
           1       0.40      0.38      0.39        21

    accuracy                           0.39        41
   macro avg       0.39      0.39      0.39        41
weighted avg       0.39      0.39      0.39        41

Potential Downside Swing Predicted for QQQ


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
